# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext(appName='sga')

from pyspark.sql import SparkSession, Row
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-01-21 11:26:52,132 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pyspark.sql.functions as f
from pyspark.sql.window import Window

In [244]:
clickstream = spark.read.csv('clickstream.csv', header=True, sep='\t') # for df 
clickstream.createOrReplaceTempView('clickstream') # for sql

In [275]:
%%markdown
# DF solution

# DF solution


In [291]:
# window to check for errors over (and within) different sessions and users
event_window = Window.partitionBy("user_id", "session_id").orderBy("timestamp")

# flagging rows that either contain the 'error' substring or follow the row with the 'error' substring 
clickstream_w_error_flags = (clickstream
                             .withColumn("error_flag", 
                                         f.max(
                                             f.when(f.col("event_type").like("%error%"), 1)
                                             .otherwise(0))
                                             .over(event_window))
                                             )

# removing errors
filtered_from_errors = clickstream_w_error_flags.filter(f.col('error_flag') != 1)

# the only way to collect routes in a parallelized way is via collect_list func.
# however, we need to deduplicate cases when consecutive rows contain the same 'event_page' value
# this is done because internet -> internet is not a valid click transition and should not be considered
filtered_w_dup_flags = (filtered_from_errors
              .withColumn("to_remove",
                    f.when(f.lag('event_page').over(event_window) == f.col('event_page'), 1)
                    .otherwise(0))
     )
deduplicated = filtered_w_dup_flags.filter(f.col('to_remove') != 1)

# creating a string of max route per user and session
deduplicated_routes = deduplicated.withColumn('sorted_route', f.concat_ws("-", f.collect_list('event_page').over(event_window)))

# finding all max routes per user session
max_routes = (deduplicated_routes
          .groupby('user_id', 'session_id')
          .agg(f.max('sorted_route').alias('route'))
         )

# finding 30 most frequent routes
routes_df = (max_routes
                   .groupby('route').agg(f.count('*').alias('count'))
                   .orderBy(f.desc('count'))
                   .limit(30)
                  )

routes_df.show(30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1113|
|         main-rabota| 1047|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  677|
|         main-online|  587|
|          main-vklad|  518|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  139|
|   main-bonus-rabota|  135|
|    main-news-rabota|  135|
|main-archive-inte...|  132|
|    main-rabota-news|  130|
|main-internet-rabota|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
| main-internet-bonus|  115|
|main-tariffs-inte...|  114|
|   main-news-archive|  113|
|  main-news-internet|  109|
|main-archive-tariffs|  104|
|  main-internet-news|  103|
|main-tariffs-archive|  103|
|    main-rabota-main|   94|
+--------------------+-----+



In [279]:
%%markdown
# SQL solution

# SQL solution


In [290]:
routes_sql = spark.sql("""
WITH error_free AS (
    SELECT 
        user_id, 
        session_id, 
        event_page, 
        timestamp,
        MAX(CASE WHEN event_type LIKE '%error%' THEN 1 ELSE 0 END) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) AS error_flag
    FROM 
        clickstream
),

duplicate_free AS (
    SELECT 
        user_id, 
        session_id,
        event_page,
        timestamp,
        CASE WHEN LAG(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) = event_page THEN 1 ELSE 0 END AS to_remove
    FROM 
        error_free
    WHERE
        error_flag <> 1
),

collected_routes AS (
    SELECT 
        user_id, 
        session_id,
        CONCAT_WS('-', COLLECT_LIST(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp)) AS sorted_route
    FROM 
        duplicate_free
    WHERE 
        to_remove = 0
    GROUP BY 
        user_id, session_id, event_page, timestamp
),

max_routes AS (
    SELECT
        user_id, 
        session_id,
        MAX(sorted_route) AS max_route
    FROM 
        collected_routes
    GROUP BY
        user_id, session_id
)

SELECT 
    max_route AS route,
    COUNT(*) as count
FROM 
    max_routes 
GROUP BY
    max_route
ORDER BY 
    count DESC
LIMIT 30

""")

routes_sql.show(30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1113|
|         main-rabota| 1047|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  677|
|         main-online|  587|
|          main-vklad|  518|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  139|
|   main-bonus-rabota|  135|
|    main-news-rabota|  135|
|main-archive-inte...|  132|
|    main-rabota-news|  130|
|main-internet-rabota|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
| main-internet-bonus|  115|
|main-tariffs-inte...|  114|
|   main-news-archive|  113|
|  main-news-internet|  109|
|main-archive-tariffs|  104|
|  main-internet-news|  103|
|main-tariffs-archive|  103|
|    main-rabota-main|   94|
+--------------------+-----+



In [282]:
%%markdown
# RDD solution

# RDD solution


In [292]:
error_sessions = (clickstream
                  .rdd
                  .filter(lambda row: 'error' in row.event_type)
                  .map(lambda row: ((row.user_id, row.session_id), row.timestamp))
                  .reduceByKey(lambda x, y: min(x, y)))

clickstream_wo_errors = (clickstream
                        .rdd
                        .map(lambda row: ((row.user_id, row.session_id), row))
                        .leftOuterJoin(error_sessions)
                        .filter(lambda row: row[1][1] is None or row[1][0].timestamp < row[1][1])
                        .map(lambda row: row[1][0]))

longest_routes = (clickstream_wo_errors
                  .map(lambda row: ((row.user_id, row.session_id), row.event_page))
                  .groupByKey()
                  .map(lambda row: (row[0], list(row[1]))))


def remove_consecutive_duplicates(lst):
    return [x for x, y in zip(lst, lst[1:] + [None]) if x != y]

longest_routes = longest_routes.map(lambda row: (row[0], '-'.join(remove_consecutive_duplicates(row[1]))))
                 
routes_rdd = (longest_routes
          .map(lambda row: (row[1], 1))
          .reduceByKey(lambda x, y: x + y)
          .sortBy(lambda row: row[1], ascending=False)
          .take(30))

routes_rdd

[('main', 8184),
 ('main-archive', 1106),
 ('main-rabota', 1045),
 ('main-internet', 894),
 ('main-bonus', 872),
 ('main-news', 764),
 ('main-tariffs', 676),
 ('main-online', 586),
 ('main-vklad', 514),
 ('main-rabota-archive', 170),
 ('main-archive-rabota', 167),
 ('main-bonus-archive', 142),
 ('main-rabota-bonus', 140),
 ('main-news-rabota', 135),
 ('main-bonus-rabota', 134),
 ('main-archive-internet', 133),
 ('main-rabota-news', 129),
 ('main-internet-rabota', 128),
 ('main-archive-news', 126),
 ('main-rabota-internet', 124),
 ('main-internet-archive', 123),
 ('main-archive-bonus', 119),
 ('main-tariffs-internet', 115),
 ('main-internet-bonus', 114),
 ('main-news-archive', 113),
 ('main-news-internet', 109),
 ('main-internet-news', 103),
 ('main-tariffs-archive', 103),
 ('main-archive-tariffs', 103),
 ('main-rabota-main', 94)]

In [296]:
# writing results

# df
routes_pd = routes_df.toPandas()
routes_pd.to_csv('routes_df.csv', sep='\t', index=False)

# sql
routes_pd = routes_sql.toPandas()
routes_pd.to_csv('routes_sql.csv', sep='\t', index=False)

# rdd
with open('routes_rdd.csv', 'w') as f:
    f.write(f'route\tcount\n')
    for route, count in routes:
        f.write(f'{route}\t{count}\n')

In [ ]:
spark.stop()